드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


패키지 설치

In [2]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import warnings 
warnings.filterwarnings(action='ignore')

경로 설정 및 파일 불러오기

In [3]:
!unzip -q '/content/drive/My Drive/open.zip' -d '/content/drive/My Drive/NLP/'

replace /content/drive/My Drive/NLP/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/NLP/test_x.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/NLP/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [4]:
train = pd.read_csv('/content/drive/My Drive/NLP/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/My Drive/NLP/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/drive/My Drive/NLP/sample_submission.csv', encoding = 'utf-8')

In [5]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [6]:
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [7]:
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


전처리

In [8]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [9]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [10]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [11]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].str.lower().apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].str.lower().apply(alpha_num).apply(remove_stopwords)

In [12]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [13]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype='<U1433')

모델링

In [14]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 64
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [15]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [16]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [17]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [18]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 64)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 1,281,685
Trainable params: 1,281,685
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
# fit model
num_epochs = 100
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/100
1372/1372 - 24s - loss: 0.4712 - accuracy: 0.8288 - val_loss: 0.8149 - val_accuracy: 0.7086
Epoch 2/100
1372/1372 - 23s - loss: 0.4575 - accuracy: 0.8345 - val_loss: 0.7996 - val_accuracy: 0.7199
Epoch 3/100
1372/1372 - 22s - loss: 0.4408 - accuracy: 0.8406 - val_loss: 0.8134 - val_accuracy: 0.7206
Epoch 4/100
1372/1372 - 21s - loss: 0.4280 - accuracy: 0.8458 - val_loss: 0.8306 - val_accuracy: 0.7220
Epoch 5/100
1372/1372 - 21s - loss: 0.4172 - accuracy: 0.8481 - val_loss: 0.8213 - val_accuracy: 0.7269
Epoch 6/100
1372/1372 - 22s - loss: 0.4064 - accuracy: 0.8534 - val_loss: 0.8437 - val_accuracy: 0.7167
Epoch 7/100
1372/1372 - 22s - loss: 0.3974 - accuracy: 0.8557 - val_loss: 0.8466 - val_accuracy: 0.7248
Epoch 8/100
1372/1372 - 22s - loss: 0.3876 - accuracy: 0.8587 - val_loss: 0.8680 - val_accuracy: 0.7249
Epoch 9/100
1372/1372 - 23s - loss: 0.3818 - accuracy: 0.8617 - val_loss: 0.8714 - val_accuracy: 0.7200
Epoch 10/100
1372/1372 - 22s - loss: 0.3714 - accuracy: 0.8662 -

In [21]:
# predict values
pred = model.predict_proba(test_padded)

Instructions for updating:
Please use `model.predict()` instead.


In [22]:
pred

array([[1.9217322e-11, 1.0657001e-03, 3.6651248e-01, 6.3242185e-01,
        1.0029633e-10],
       [5.6270674e-06, 2.2112215e-03, 9.3159889e-04, 9.9684441e-01,
        7.1080781e-06],
       [1.0000000e+00, 7.1525151e-16, 4.5353072e-18, 3.2461979e-19,
        3.8889053e-15],
       ...,
       [6.1654626e-13, 1.0000000e+00, 1.9514051e-13, 3.1563524e-13,
        6.4095347e-18],
       [6.5121075e-11, 1.0000000e+00, 4.0540352e-13, 3.4949284e-14,
        2.8960204e-11],
       [1.0000000e+00, 8.5089526e-19, 2.2025474e-17, 3.6213160e-17,
        1.4860668e-16]], dtype=float32)

제출

In [23]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,1.921732e-11,1.065700e-03,3.665125e-01,6.324219e-01,1.002963e-10
1,1,5.627067e-06,2.211221e-03,9.315989e-04,9.968444e-01,7.108078e-06
2,2,1.000000e+00,7.152515e-16,4.535307e-18,3.246198e-19,3.888905e-15
3,3,2.225493e-16,5.429704e-32,9.999605e-01,8.120219e-19,3.940853e-05
4,4,1.244883e-02,4.911875e-16,8.761579e-08,9.875512e-01,2.146003e-16
...,...,...,...,...,...,...
19612,19612,4.287311e-07,9.999995e-01,2.082235e-27,2.034445e-16,3.750827e-28
19613,19613,2.584100e-06,8.309962e-24,2.011517e-21,6.792680e-37,9.999974e-01
19614,19614,6.165463e-13,1.000000e+00,1.951405e-13,3.156352e-13,6.409535e-18
19615,19615,6.512108e-11,1.000000e+00,4.054035e-13,3.494928e-14,2.896020e-11


In [24]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')